In [349]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%run Data_Center_setup.ipynb

from collections import defaultdict

import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum

print(adjacent_zone_map)
print(T_zone_initial.reshape(1, -1))

[[0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0.]]
[[31 30 29 32 26 26 30 24 27]]


$$
\begin{equation*}
\begin{array}{lll}
&\displaystyle \min_{\substack{
u^{\text{dis}}_{t},\ u^{\text{chr}}_{t},p^{\text{dis}}_{t},\ p^{\text{chr}}_{t},x^{\text{chiller}}_{j,t},\ x^{\text{tower}}_{j,t} \\
T_{i,t}^{\text{Zone}},\ T_{i,t+1}^{\text{Zone}} ,T_{i,t}^{\text{sup}},\ v^{\text{vent}}_{t} \\}} 
\sum_{t=0}^{T}p^{e,g}_{t} e^{\text{dc,in}}_{t} & \\
s.t.\quad &e^{\text{dc,in}}_{t} = E^{\text{HVAC}}_t + E^{\text{Se}}_{t} +\Delta E^{\text{B}}_{t} - E^{\text{Solar}}_{t} & \forall  t \in \mathbf{T}\\
&e^{\text{dc,in}}_{t} \geq 0 & \forall t \in \mathbf{T}\\
& \Delta E^{\text{B}}_{t} = p^{\text{chr}}_{t}\eta^{\text{chr}} - p^{\text{dis}}_{t} \cdot (\eta^{\text{dis}})^{-1} & \forall  t \in \mathbf{T}\\ 
& E^{\text{B,state}}_{t+1} = E^{\text{B,state}}_{t} + \Delta E^{\text{B}}_{t} & \forall  t \in \mathbf{T} \\
& \underline{\xi^{\text{B}}} \leq E^{\text{B,state}}_{t} \leq \overline{\xi^{\text{B}}} & \forall  t \in \mathbf{T} \\
& p^{\text{chr}}_{t} \leq \overline{p^{\text{chr}}_{t}} \cdot u^{\text{chr}}_{t} & \forall  t \in \mathbf{T}\\
& p^{\text{dis}}_{t} \leq \overline{p^{\text{dis}}_{t}} \cdot u^{\text{dis}}_{t} & \forall  t \in \mathbf{T}\\
& u^{\text{chr}}_{t} + u^{\text{dis}}_{t} \leq 1 & \forall  t \in \mathbf{T}\\
&T^{\text{Zone}}_{i,t+1} = T^{\text{Zone}}_{i,t} 
+ \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)} 
\left(\frac{T_{i^{\prime},t}^{\text{Zone}} - T_{i,t}^{\text{Zone}}}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right)
+ \frac{\dot{m}_{i} c_{a}\left(T_{i,t}^{\text{sup}} - T_{i,t}^{\text{Zone}}\right) 
+ \theta_{i,t}}{C^{\text{heat}}_{i}} & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T}\\
&T_{i,t}^{-} \leq T^{\text{Zone}}_{i,t} \leq T_{i,t}^{+}, & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T} \\
&v^{\text{vent}}_t + v^{\text{out}}_t \geq \underline{v}^{\text{vent}}_t & \forall  t \in \mathbf{T}\\
&L^{\text{heat}}_{t} = \left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{chiller}}} x^{\text{chiller}}_{j,t}m^{\text{chiller}}_{j,t} \geq \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{tower}}}{x^{\text{tower}}_{j,t}m^{\text{tower}}_{j,t}} \geq \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p }
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot  c^{\text{water}}_p }  & \forall t \in \mathbf{T}\\
&e^{\text{vent}}_{t}=\beta^{\text{vent}}_{0}+\beta^{\text{vent}}_{1}\left(v^{\text{vent}}_t-\underline{v}^{\text{vent}}\right) 
& \forall t \in \mathbf{T}\\
&e^{\text{chiller}}_{t}=\displaystyle\sum_{j \in \mathbf{J}^{\text{chiller}}}x^{\text{chiller}}_{j,t}\left(\beta^{\text{chiller}}_{0,j}
 + \beta^{\text{chiller}}_{1,j}m^{\text{chiller}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&e^{\text {pump }}_{t}=\beta^{\text{pump}}_{0}+\beta^{\text{pump}}_{1} \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&e^{\text{tower}}_{t} = \displaystyle\sum_{j \in \mathbf{J}^{\text{tower}}}x^{\text{tower}}_{j,t} \left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j}m^{\text{tower}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&E^{\text{HVAC}}_{t} =  e^{\text{sup}}_{t} + e^{\text{vent}}_{t}+ e^{\text{chiller}}_{t} + e^{\text{pump}}_{t} + e^{\text{tower}}_{t} & \forall  t \in \mathbf{T}\\
\end{array}
\end{equation*}
$$

In [350]:

p_eg

array([1, 2, 3, 4])

In [460]:
%run Data_Center_setup.ipynb
# create one model instance, with a name
DCMG = gp.Model("Data_center_model")

# continuous decision variables
# charging and discharging of battery rate
p_char = DCMG.addVars(T, vtype=GRB.CONTINUOUS, name='p_char')
p_dis = DCMG.addVars(T, vtype=GRB.CONTINUOUS, name='p_char')

# zone i AC temperature
Temp_AC = DCMG.addVars(I_zone, T, vtype=GRB.CONTINUOUS, name='Temp_sup')

# ventilation air flow rate
v_vent = DCMG.addVars(T, vtype=GRB.CONTINUOUS, name='v_vent')

# continuous state variables
# room air temperature
Temp_Zone = DCMG.addVars(I_zone, T + 1, vtype=GRB.CONTINUOUS, name='Temp_zone')

# battery state 
Battery_state = DCMG.addVars(T + 1, vtype=GRB.CONTINUOUS, name='Battery_state')

# eletricity demand
e_in = DCMG.addVars(T, vtype=GRB.CONTINUOUS, name='e_in')

# binary continuous variables
# charging and discharging of battery state
u_char = DCMG.addVars(T, vtype=GRB.BINARY, name='u_char')
u_dis = DCMG.addVars(T, vtype=GRB.BINARY, name='u_dis')

# chiller and tower on/off state
x_chiller = DCMG.addVars(J_chiller, T, vtype=GRB.BINARY, name='x_chiller')
x_tower = DCMG.addVars(J_tower, T, vtype=GRB.BINARY, name='x_tower')

# objective function
DCMG.setObjective(quicksum( e_in[t] * p_eg[t] for t in range(T)), GRB.MINIMIZE)

# constarint 1
for t in range(T):
    DCMG.addConstr(e_in[t] == e_server[t] + p_char[t] * eta_char_b - p_dis[t] * eta_dischar_b - e_solar[t] + e_sup[t] + \
                      beta_vent_0 + beta_vent_1 * (v_vent[t] - v_vent_startup)  + \
                      quicksum(x_chiller[j,t] * (beta_chiller_0[j] +  beta_chiller_1[j] * M_chiller_rated[j]) for j in range(J_chiller)) + \
                      quicksum(x_tower[j,t] * (beta_tower_0[j] +  beta_tower_1[j] * M_tower_rated[j]) for j in range(J_tower)) + \
                      beta_pump_0 + beta_pump_1 / (C_water * (T_chiller_water_return[t] - T_chiller_water_supply[t])) * \
                      ((T_out[t] -  quicksum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone)) ) * v_out[t] * c_s_air  + \
                      quicksum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * c_s_air)  )  
    
# constarint 2
for t in range(T):
    DCMG.addConstr(e_in[t] >= 0)

# constarint 3
DCMG.addConstr(Battery_state[0] == 0.5 * Battery_capacity)
   
# constarint 3.5
for t in range(T):
    DCMG.addConstr(Battery_state[t + 1] == Battery_state[t] + p_char[t] * eta_char_b - p_dis[t] * eta_dischar_b)
    DCMG.addConstr(Battery_state[t + 1] >= Battery_capacity_lowbound)
    DCMG.addConstr(Battery_state[t + 1] <= Battery_capacity_upbound)
    
    
# constarint 4 & 5
for t in range(T):
    DCMG.addConstr(p_char[t] <= char_b_upperbound * u_char[t])
    DCMG.addConstr(p_dis[t] <= dis_b_upperbound * u_dis[t])

# constarint 6
for t in range(T):
    DCMG.addConstr(u_char[t] + u_dis[t] <= 1)
     
# constarint 7 & 7.5 
for i in range(I_zone):
    #print(i // I_zone_length, i % I_zone_length)
    DCMG.addConstr( Temp_Zone[i,0] == T_zone_initial [i // I_zone_length, i % I_zone_length] )  # unit: C
    #print(T_zone_initial)
    
for i in range(I_zone):
    for t in range(T):
        DCMG.addConstr( Temp_Zone[i, t + 1] ==  Temp_Zone[i,t] + quicksum(adjacency * (Temp_Zone[node,t] - Temp_Zone[i,t])  for node, adjacency in enumerate(adjacent_zone_map[i,:]) if node != i)/\
        (C_air_room[i // I_zone_length, i % I_zone_length] * R_air_ij) + (m_air_in[i,t] * c_s_air * (Temp_AC[i,t] - Temp_Zone[i,t]) + theta_heat[i,t]) / (C_air_room[i // I_zone_length, i % I_zone_length]) )


# constarint 8 & 8.5
for t in range(1, T + 1):
    for i in range(I_zone):
        DCMG.addConstr(Temp_Zone[i,t] >= T_datacenter_lowerbound[i,t-1])
        DCMG.addConstr(Temp_Zone[i,t] <= T_datacenter_upperbound[i,t-1])
        

# constarint 9 & 9.5
for t in range(T):
    DCMG.addConstr(v_vent[t] + v_out[t] >= v_vent_lowerbound[t])

# constarint 10
for t in range(T):
    DCMG.addConstr(quicksum( x_chiller[j,t] * M_chiller_rated[j] for j in range(J_chiller)) >= \
    ((T_out[t] -  quicksum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone))) * v_out[t] * c_s_air + \
    quicksum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * c_s_air)/ \
    (T_chiller_water_return[t] - T_chiller_water_supply[t])/C_water)
    
# constarint 11 
for t in range(T):
    DCMG.addConstr(quicksum( x_tower[j,t] * M_tower_rated[j] for j in range(J_tower)) >= \
    ((T_out[t] -  quicksum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone))) * v_out[t] * c_s_air + \
    quicksum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * c_s_air)/ \
    (T_tower_water_return[t] - T_tower_water_supply[t])/C_water )

    
# constarint 12(added)
for i in range(I_zone):
    for t in range(T):
        DCMG.addConstr(Temp_AC[i,t] >= T_AC_lowerbound[i,t])
        DCMG.addConstr(Temp_AC[i,t] <= T_AC_upperbound[i,t])

DCMG_sol = DCMG.optimize()
#assert DCMG_sol
#DCMG.display()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads
Optimize a model with 234 rows, 162 columns and 642 nonzeros
Model fingerprint: 0x4de494a4
Variable types: 102 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [3e-03, 2e+02]
  Objective range  [2e+02, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 205 rows and 107 columns
Presolve time: 0.00s
Presolved: 29 rows, 55 columns, 195 nonzeros
Variable types: 50 continuous, 5 integer (4 binary)
Found heuristic solution: objective 1.507780e+07

Root relaxation: objective 1.464232e+07, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.4642e+07    0    3 1.5078e+07 1.4642e+07  2.89%     -    0s
H    0     0                    1.48

In [464]:
u_sol = DCMG.getVars()
print('Obj: %g' % DCMG.ObjVal)

print(DCMG.getAttr(GRB.Attr.Status))

Obj: 1.48047e+07
2


In [458]:
%run Data_Center_setup.ipynb
'''
t = 0

print(C_air_room)
print(T_zone_initial)

for i in range(9):
    print(T_zone_initial[i // I_zone_length, i % I_zone_length] + \
        sum(adjacency * (T_zone_initial[node // I_zone_length , node % I_zone_length] -\
        T_zone_initial[i // I_zone_length, i % I_zone_length])  for node, adjacency in enumerate(adjacent_zone_map[i,:]) if node != i)/(C_air_room[i // I_zone_length, i % I_zone_length] * R_air_ij) 
        +
        (m_air_in[i,t] * c_s_air * (12 - T_zone_initial[i // I_zone_length, i % I_zone_length]) + theta_heat[i,t]) / (C_air_room[i // I_zone_length, i % I_zone_length]))
    
    #print()
''' 
    
 

'\nt = 0\n\nprint(C_air_room)\nprint(T_zone_initial)\n\nfor i in range(9):\n    print(T_zone_initial[i // I_zone_length, i % I_zone_length] +         sum(adjacency * (T_zone_initial[node // I_zone_length , node % I_zone_length] -        T_zone_initial[i // I_zone_length, i % I_zone_length])  for node, adjacency in enumerate(adjacent_zone_map[i,:]) if node != i)/(C_air_room[i // I_zone_length, i % I_zone_length] * R_air_ij) \n        +\n        (m_air_in[i,t] * c_s_air * (12 - T_zone_initial[i // I_zone_length, i % I_zone_length]) + theta_heat[i,t]) / (C_air_room[i // I_zone_length, i % I_zone_length]))\n    \n    #print()\n'

In [459]:
#(m_air_in[i,t] * c_s_air * (26 - T_zone_initial[i // I_zone_length, i % I_zone_length]) + theta_heat[i,t]) / (C_air_room[i // I_zone_length, i % I_zone_length])

In [380]:
#t = 0
#((T_out[t] -  sum(Chi[t] * T_zone_initial[i // I_zone_length, i % I_zone_length] for i in range(I_zone))) * v_out[t] * c_s_air + \
#    sum(Chi[i] * (T_zone_initial[i // I_zone_length, i % I_zone_length] - 26) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * c_s_air)/ \
#    ((T_tower_water_return[t] - T_tower_water_supply[t])*C_water)

63.39256152742522

In [381]:
##((T_out[t] -  sum(Chi[t] * T_zone_initial[i // I_zone_length, i % I_zone_length] for i in range(I_zone))) * v_out[t] * c_s_air + \
#    sum(Chi[i] * (T_zone_initial[i // I_zone_length, i % I_zone_length] - 26) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * c_s_air)/ \
#    ((T_chiller_water_return[t] - T_chiller_water_supply[t])*C_water)

82.41032998565278

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads
Optimize a model with 234 rows, 162 columns and 642 nonzeros
Model fingerprint: 0xeec75a22
Variable types: 102 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [3e-03, 2e+02]
  Objective range  [2e+02, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 199 rows and 100 columns
Presolve time: 0.00s
Presolved: 35 rows, 62 columns, 224 nonzeros
Variable types: 54 continuous, 8 integer (2 binary)
Found heuristic solution: objective 1.298699e+07

Root relaxation: objective 1.283252e+07, 39 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.2833e+07    0    5 1.2987e+07 1.2833e+07  1.19%     -    0s
     0     0 1.2987e+07    0    4 1.

In [455]:
u_sol = DCMG.getVars()
print('Obj: %g' % DCMG.ObjVal)

print(DCMG.getAttr(GRB.Attr.Status))

x_tower

Obj: 1.2987e+07
2


{(0, 0): <gurobi.Var x_tower[0,0] (value 1.0)>,
 (0, 1): <gurobi.Var x_tower[0,1] (value 0.0)>,
 (0, 2): <gurobi.Var x_tower[0,2] (value 0.0)>,
 (0, 3): <gurobi.Var x_tower[0,3] (value 0.0)>,
 (1, 0): <gurobi.Var x_tower[1,0] (value 1.0)>,
 (1, 1): <gurobi.Var x_tower[1,1] (value 0.0)>,
 (1, 2): <gurobi.Var x_tower[1,2] (value 1.0)>,
 (1, 3): <gurobi.Var x_tower[1,3] (value 0.0)>,
 (2, 0): <gurobi.Var x_tower[2,0] (value 0.0)>,
 (2, 1): <gurobi.Var x_tower[2,1] (value 1.0)>,
 (2, 2): <gurobi.Var x_tower[2,2] (value 0.0)>,
 (2, 3): <gurobi.Var x_tower[2,3] (value 1.0)>,
 (3, 0): <gurobi.Var x_tower[3,0] (value 0.0)>,
 (3, 1): <gurobi.Var x_tower[3,1] (value 0.0)>,
 (3, 2): <gurobi.Var x_tower[3,2] (value 0.0)>,
 (3, 3): <gurobi.Var x_tower[3,3] (value 1.0)>,
 (4, 0): <gurobi.Var x_tower[4,0] (value 0.0)>,
 (4, 1): <gurobi.Var x_tower[4,1] (value 0.0)>,
 (4, 2): <gurobi.Var x_tower[4,2] (value 0.0)>,
 (4, 3): <gurobi.Var x_tower[4,3] (value 0.0)>,
 (5, 0): <gurobi.Var x_tower[5,0] (value

In [465]:

for v in u_sol:
    if v.VarName.split("_")[0] == "x":
        print('%s %g' % (v.VarName, v.X))



x_chiller[0,0] 1
x_chiller[0,1] 1
x_chiller[0,2] 1
x_chiller[0,3] 1
x_chiller[1,0] 1
x_chiller[1,1] 1
x_chiller[1,2] 1
x_chiller[1,3] -0
x_chiller[2,0] 1
x_chiller[2,1] 1
x_chiller[2,2] -0
x_chiller[2,3] 1
x_chiller[3,0] 1
x_chiller[3,1] 1
x_chiller[3,2] 1
x_chiller[3,3] 1
x_chiller[4,0] 1
x_chiller[4,1] 1
x_chiller[4,2] 1
x_chiller[4,3] 1
x_tower[0,0] 1
x_tower[0,1] 0
x_tower[0,2] 1
x_tower[0,3] 0
x_tower[1,0] 0
x_tower[1,1] 0
x_tower[1,2] 0
x_tower[1,3] 0
x_tower[2,0] 0
x_tower[2,1] 1
x_tower[2,2] 0
x_tower[2,3] 1
x_tower[3,0] 0
x_tower[3,1] 0
x_tower[3,2] 1
x_tower[3,3] 0
x_tower[4,0] 1
x_tower[4,1] 0
x_tower[4,2] 1
x_tower[4,3] 0
x_tower[5,0] 0
x_tower[5,1] 1
x_tower[5,2] 0
x_tower[5,3] 0
x_tower[6,0] 0
x_tower[6,1] 1
x_tower[6,2] 0
x_tower[6,3] 1
x_tower[7,0] 1
x_tower[7,1] 0
x_tower[7,2] 0
x_tower[7,3] 1


In [466]:
for v in u_sol:
    if v.VarName.split("_")[0] == "Temp":
        print('%s %g' % (v.VarName, v.X))

Temp_sup[0,0] 11.3625
Temp_sup[0,1] 16.2625
Temp_sup[0,2] 16.2625
Temp_sup[0,3] 16.2625
Temp_sup[1,0] 12.7833
Temp_sup[1,1] 16.2625
Temp_sup[1,2] 16.2809
Temp_sup[1,3] 16.275
Temp_sup[2,0] 13.3562
Temp_sup[2,1] 16.2625
Temp_sup[2,2] 16.2675
Temp_sup[2,3] 16.2625
Temp_sup[3,0] 10.8426
Temp_sup[3,1] 16.2625
Temp_sup[3,2] 16.2809
Temp_sup[3,3] 16.275
Temp_sup[4,0] 14.6875
Temp_sup[4,1] 10
Temp_sup[4,2] 14.668
Temp_sup[4,3] 18.0619
Temp_sup[5,0] 14.775
Temp_sup[5,1] 14.5841
Temp_sup[5,2] 16.9759
Temp_sup[5,3] 16.275
Temp_sup[6,0] 13.2833
Temp_sup[6,1] 16.2625
Temp_sup[6,2] 16.2811
Temp_sup[6,3] 16.2625
Temp_sup[7,0] 16.2053
Temp_sup[7,1] 10
Temp_sup[7,2] 18.909
Temp_sup[7,3] 16.275
Temp_sup[8,0] 14.6333
Temp_sup[8,1] 16.2625
Temp_sup[8,2] 16.2861
Temp_sup[8,3] 16.2625
Temp_zone[0,0] 31
Temp_zone[0,1] 24
Temp_zone[0,2] 24
Temp_zone[0,3] 24
Temp_zone[0,4] 24
Temp_zone[1,0] 30
Temp_zone[1,1] 24
Temp_zone[1,2] 24
Temp_zone[1,3] 24
Temp_zone[1,4] 24
Temp_zone[2,0] 29
Temp_zone[2,1] 24
Temp_zone